In [34]:
import csv
import numpy as np
import urllib.request
import io

Читаем csv с векторами предложений. Один файл - либо 1, либо 2 предложение одной из частей данных

In [2]:
def read_csv(filename):
    with open(filename) as csvfile:
        spamreader = csv.reader(csvfile)
        vectors = []
        i = 0
        for row in spamreader:
            if i == 0:
                i += 1
                continue
            row = [float(el) for el in row[1:]]
            vectors.append(row)
    return vectors

In [25]:
def read_csv_link(response):
    spamreader = csv.reader(response)
    vectors = []
    i = 0
    for row in spamreader:
        if i == 0:
            i += 1
            continue
        row = [float(el) for el in row[1:]]
        vectors.append(row)
    return vectors

Считаем расстояние между векторами предложений, здесь тоже просто вычитается один из другого, мб можно попробовать другие метрики

In [3]:
def count_dist(file1, file2):
    vectors_1 = read_csv(file1)
    vectors_2 = read_csv(file2)
    vectors_1 = np.array([np.array(vectors_1[i]) for i in range(len(vectors_1))])
    vectors_2 = np.array([np.array(vectors_2[i]) for i in range(len(vectors_2))])
    dist_1 = vectors_1 - vectors_2
    return dist_1

In [26]:
def count_dist_link(file1, file2):
    vectors_1 = read_csv_link(file1)
    vectors_2 = read_csv_link(file2)
    vectors_1 = np.array([np.array(vectors_1[i]) for i in range(len(vectors_1))])
    vectors_2 = np.array([np.array(vectors_2[i]) for i in range(len(vectors_2))])
    dist_1 = vectors_1 - vectors_2
    return dist_1

Если есть файлы

In [4]:
X_train = count_dist('train_1.csv', 'train_2.csv')
X_val = count_dist('val_1.csv', 'val_2.csv')
X_test = count_dist('test_1.csv', 'test_2.csv')

Скачать с гита

In [36]:
response_val_1_1 = urllib.request.urlopen('https://raw.githubusercontent.com/ulianakazak/Bigutochki/main/val_1_1.csv')
response_val_2_1 = urllib.request.urlopen('https://raw.githubusercontent.com/ulianakazak/Bigutochki/main/val_2_1.csv')
response_val_1_2 = urllib.request.urlopen('https://raw.githubusercontent.com/ulianakazak/Bigutochki/main/val_1_2.csv')
response_val_2_2 = urllib.request.urlopen('https://raw.githubusercontent.com/ulianakazak/Bigutochki/main/val_2_2.csv')
response_test_1_1 = urllib.request.urlopen('https://raw.githubusercontent.com/ulianakazak/Bigutochki/main/test_1_1.csv')
response_test_2_1 = urllib.request.urlopen('https://raw.githubusercontent.com/ulianakazak/Bigutochki/main/test_2_1.csv')
response_test_1_2 = urllib.request.urlopen('https://raw.githubusercontent.com/ulianakazak/Bigutochki/main/test_1_2.csv')
response_test_2_2 = urllib.request.urlopen('https://raw.githubusercontent.com/ulianakazak/Bigutochki/main/test_2_2.csv')

X_val_1 = count_dist_link(io.TextIOWrapper(response_val_1_1), io.TextIOWrapper(response_val_2_1))
X_val_2 = count_dist_link(io.TextIOWrapper(response_val_1_2), io.TextIOWrapper(response_val_2_2))
X_val = np.vstack((X_val_1, X_val_2))

X_test_1 = count_dist_link(io.TextIOWrapper(response_test_1_1), io.TextIOWrapper(response_test_2_1))
X_test_2 = count_dist_link(io.TextIOWrapper(response_test_1_2), io.TextIOWrapper(response_test_2_2))
X_test = np.vstack((X_test_1, X_test_2))

In [40]:
response_1 = urllib.request.urlopen('https://raw.githubusercontent.com/ulianakazak/Bigutochki/main/data_syntax/train_1_0.csv')
response_2 = urllib.request.urlopen('https://raw.githubusercontent.com/ulianakazak/Bigutochki/main/data_syntax/train_2_0.csv')
X_train = count_dist_link(io.TextIOWrapper(response_1), io.TextIOWrapper(response_2))
for i in range(1, 68):
    response_1 = urllib.request.urlopen(f'https://raw.githubusercontent.com/ulianakazak/Bigutochki/main/data_syntax/train_1_{i}.csv')
    response_2 = urllib.request.urlopen(f'https://raw.githubusercontent.com/ulianakazak/Bigutochki/main/data_syntax/train_2_{i}.csv')
    X = count_dist_link(io.TextIOWrapper(response_1), io.TextIOWrapper(response_2))
    X_train = np.vstack((X_train, X))

In [ ]:
len(X_train)

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import fbeta_score,make_scorer

In [8]:
train = pd.read_json('snli_1.0_train.jsonl', lines = True)
test = pd.read_json('snli_1.0_test.jsonl', lines = True)
val = pd.read_json('snli_1.0_dev.jsonl', lines = True)
full = {'train': train, 'val':val, 'test': test}

In [9]:
def delete_unsure_labels(data):
    data = data.drop(index = data[data.gold_label == '-'].index)
    return data


for frame in full:
    full[frame] = delete_unsure_labels(full[frame])

In [10]:
def delete_unsure_labels_vec(X, data):
    X = X[data.index]
    return X

In [11]:
train = full['train']
val = full['val']
test = full['test']

In [13]:
X_train = delete_unsure_labels_vec(X_train, train)
X_val = delete_unsure_labels_vec(X_val, val)
X_test = delete_unsure_labels_vec(X_test, test)

In [15]:
y_train = np.array(train['gold_label'])
y_val = np.array(val['gold_label'])
y_test = np.array(test['gold_label'])

In [17]:
clf = LogisticRegression(max_iter=500, solver = 'saga')
clf.fit(X_train, y_train)
y_train_preds = clf.predict(X_train)
f_train = fbeta_score(y_train, y_train_preds, beta=7, average=None,)
y_val_preds = clf.predict(X_val)
f_val = fbeta_score(y_val, y_val_preds, beta=7, average=None,)
print(f'train: {f_train}')
print(f'val: {f_val}')

train: [0.35201383 0.38617524 0.28150132]
val: [0.83044389 0.12743774 0.01795841]


In [16]:
clf = LogisticRegression(max_iter=500, solver = 'saga')
ftwo_scorer = make_scorer(fbeta_score, beta=3, average='weighted')
cross_val_score(clf, X_test, y_test, cv=3, scoring= ftwo_scorer)

array([0.32906267, 0.33254567, 0.33654544])

In [15]:
clf = LogisticRegression(max_iter=500, solver = 'saga')
clf.fit(X_test, y_test)
y_train_preds = clf.predict(X_test)
f_train = fbeta_score(y_test, y_train_preds, beta=7, average=None,)
y_val_preds = clf.predict(X_val)
f_val = fbeta_score(y_val, y_val_preds, beta=7, average=None,)
print(f'train: {f_train}')
print(f'val: {f_val}')

train: [0.09069626 0.78048526 0.16737006]


In [18]:
prfs = precision_recall_fscore_support(y_val, y_val_preds, beta=7, average=None, labels=['contradiction', 'neutral', 'entailment'])
matrix_report = pd.DataFrame(prfs, columns = ['contradiction', 'neutral', 'entailment'], index = ['precision', 'recall', 'fscore', 'support'])
matrix_report['Weighted*'] = 0.5 * matrix_report['contradiction'] +  0.25 * matrix_report['neutral'] +  0.25 * matrix_report['entailment']
matrix_report.T

,precision,recall,fscore,support
contradiction,0.334804,0.856315,0.830444,3278.0
neutral,0.308108,0.017620,0.017958,3235.0
entailment,0.329144,0.125864,0.127438,3329.0
Weighted*,0.326715,0.464028,0.451571,3280.0
